# Quick glimpse viewer

In [27]:
import napari
import pandas as pd
import os, glob
import zarr
from tqdm.auto import tqdm
from macrohet import dataio, tile, visualise, notify
from datetime import datetime
current_date = datetime.now().strftime('%d-%m-%Y')
import btrack 
import cv2
import re
import time
import numpy as np

In [28]:
sc_df_fn = f'/mnt/SYNO/macrohet_syno/manuscript/results/sc_df.pkl'

In [29]:
df = pd.read_pickle(sc_df_fn)

In [30]:
df

,Time (hours),Mtb Area (µm),dMtb Area (µm),Mphi Area (µm),dMphi Area (µm),Infection Status,Initial Infection Status,Final Infection Status,x,y,...,Technical Replicate,Biological Replicate,Strain,Compound,Concentration,Cell ID,Acquisition ID,Experiment ID,Unique ID,ID
0,0.0,0.424621,-0.424621,459.082108,-92.746118,1.0,1.0,0.0,457.172943,8.791715,...,1,1,RD1,CTRL,EC0,426,"(3, 4)",PS0000,426.3.4,426.3.4.PS0000
1,1.0,0.000000,-0.424621,372.839393,-92.746118,0.0,1.0,0.0,459.138947,6.718642,...,1,1,RD1,CTRL,EC0,426,"(3, 4)",PS0000,426.3.4,426.3.4.PS0000
2,2.0,0.000000,-0.424621,423.168130,-92.746118,0.0,1.0,0.0,460.555237,10.785886,...,1,1,RD1,CTRL,EC0,426,"(3, 4)",PS0000,426.3.4,426.3.4.PS0000
3,3.0,0.335227,-0.424621,445.561289,-92.746118,1.0,1.0,0.0,455.878815,11.143067,...,1,1,RD1,CTRL,EC0,426,"(3, 4)",PS0000,426.3.4,426.3.4.PS0000
4,4.0,0.000000,-0.424621,487.040034,-92.746118,1.0,1.0,0.0,462.634186,14.050420,...,1,1,RD1,CTRL,EC0,426,"(3, 4)",PS0000,426.3.4,426.3.4.PS0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722130,74.5,1.117423,1.095075,752.584461,55.088959,1.0,1.0,1.0,227.093262,130.358139,...,2,3,RD1,BDQ,EC99,1874,"(6, 12)",ND0003,1874.6.12,1874.6.12.ND0003
1722131,75.0,2.033710,1.095075,616.035357,55.088959,1.0,1.0,1.0,226.622925,130.532120,...,2,3,RD1,BDQ,EC99,1874,"(6, 12)",ND0003,1874.6.12,1874.6.12.ND0003
1722132,75.5,1.430302,1.095075,579.182743,55.088959,1.0,1.0,1.0,225.366608,129.932709,...,2,3,RD1,BDQ,EC99,1874,"(6, 12)",ND0003,1874.6.12,1874.6.12.ND0003
1722133,76.0,1.989013,1.095075,679.147414,55.088959,1.0,1.0,1.0,226.424683,128.489410,...,2,3,RD1,BDQ,EC99,1874,"(6, 12)",ND0003,1874.6.12,1874.6.12.ND0003


In [32]:
crop_size = 750
track_scale_factor = 5.04
mask_outline = True
pattern = r"cell_ID\.(.*?)_dt"
pattern = r"cell_ID\.(.*?).mp4"
seg_fn = []
gfp_channel = 1
rfp_channel = 0
image_scale_m_per_pixel = 1.4949402023919043e-07
image_scale_um_per_pixel = image_scale_m_per_pixel*1E6
add_labels = True
boundary_buffer = 150
image_width = image_height = 6048

In [33]:
pattern = r"cell_ID\.(.*?)_dt"
# fns = glob.glob('/mnt/SYNO/macrohet_syno/manuscript/results/dt_dev_09-04-2024/glimpses/*.mp4')
# pattern = r"cell_ID\.(.*?).mp4"

previous_files = set() 

while True:
    print('Checking for new files')
    # Find all PNG files matching the pattern
    fns = glob.glob(f'/mnt/SYNO/macrohet_syno/manuscript/results/dt_dev/dt_dev_19-04-2024/inaccurate/*.png')
    new_files = set(fns) - previous_files  # Get the set difference to find new files
    
    for fn in new_files:
        ID = re.search(pattern, fn).group(1)
        # Add your processing logic here
 
    # for fn in tqdm(fns):
    #     ID = re.search(pattern, fn).group(1)
        # ID = '407.3.12.ND0003'
        output_dirname = '/mnt/SYNO/macrohet_syno/manuscript/results/dt_dev/glimpses' #os.path.dirname(fn)
        output_file = os.path.join(output_dirname, f'glimpses/cell_ID.{ID}.mp4')
        if os.path.exists(output_file):
            continue
        # set empty image array 
        rgb_stack = []
        
        # get single cell information 
        sc_df = df[df['ID']==ID]
    
        # Scale the x and y coordinates
        scaled_x_coords = sc_df.x * track_scale_factor
        scaled_y_coords = sc_df.y * track_scale_factor
        
        # if (any(scaled_x_coords < boundary_buffer) or 
        #     any(scaled_y_coords < boundary_buffer) or
        #     any(scaled_x_coords > image_width-boundary_buffer) or 
        #     any(scaled_y_coords > image_height-boundary_buffer)):
        #     # Save empty text file with "boundary_case" at the beginning
        #     boundary_file = os.path.join(output_dirname, f'glimpses/boundary_case_cell_ID_{ID}.txt')
        #     with open(boundary_file, 'w') as f:
        #         f.write("boundary_case\n")
        #     continue
        
        acq_ID = sc_df['Acquisition ID'].iloc[0] #  tuple(int(x) for x in ID.split('.')[1:3])
        expt_ID = sc_df['Experiment ID'].iloc[0] #ID.split('.')[-1]
        base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}'
        # convert to frames
        time_values = sc_df['Time (hours)']
        if expt_ID != 'PS0000':
            frames = time_values*2 # if expt_ID != 'PS0000' else time_values
        else:
            frames = time_values.values
        # check channel enum
        gfp_channel = 0 if expt_ID == 'PS0000' else 1
        rfp_channel = 1 if expt_ID == 'PS0000' else 0
        
        # load images
        image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
        zarr_group = zarr.open(image_dir, mode='r')
        images = zarr_group.images
        
        # check if segmentation already loaded
        if str(acq_ID) not in seg_fn:
            # define seg path
            if expt_ID == 'PS0000':
                seg_fn = os.path.join(base_dir, f'labels/macrohet_seg_model/{acq_ID}.h5')
            else:
                seg_fn = os.path.join(base_dir, f'labels/cpv3/{acq_ID}.h5')
            # load segmentation
            with btrack.io.HDF5FileHandler(seg_fn,
                                           'r',
                                           obj_type='obj_type_1') as reader:
                segmentation = reader.segmentation
        
        # iterate over time values creating
        for t, frame_number in tqdm(zip(time_values, frames), total = len(frames)):
            # # set to int
            # t = int(t)
            frame_number = int(frame_number)
            # get precise time point information 
            sc_df_t = sc_df[sc_df['Time (hours)'] == t]
            # Extract xy coordinates and transpose for python and area from the cell information
            y_coord, x_coord, area, t = sc_df_t.loc[:, ['x', 'y', 'Mphi Area (µm)', 'Time (hours)']].values[0]
            # Scale according to tracking shrinkage
            y_coord, x_coord = y_coord * track_scale_factor, x_coord * track_scale_factor
       
            if not crop_size:
                # Calculate the side length for cropping based on the square root of the area
                crop_size = int(np.sqrt(area)) * 2
        
            # Calculate the cropping boundaries
            x_start = int(x_coord - crop_size / 2)
            x_end = int(x_coord + crop_size / 2)
            y_start = int(y_coord - crop_size / 2)
            y_end = int(y_coord + crop_size / 2)
            # Pad the boundaries if they exceed the image dimensions
            if x_start < 0:
                x_pad = abs(x_start)
                x_start = 0
            else:
                x_pad = 0
        
            if x_end > images.shape[-1]:
                x_pad_end = x_end - images.shape[-1]
                x_end = images.shape[-1]
            else:
                x_pad_end = 0
        
            if y_start < 0:
                y_pad = abs(y_start)
                y_start = 0
            else:
                y_pad = 0
        
            if y_end > images.shape[-2]:
                y_pad_end = y_end - images.shape[-2]
                y_end = images.shape[-2]
            else:
                y_pad_end = 0
        
            # Crop the image
            cropped_image = images[frame_number, ..., x_start:x_end, y_start:y_end]
            # max projection 
            cropped_image = np.max(images[frame_number, ..., x_start:x_end, y_start:y_end], axis = 1)
            # Pad the cropped image if necessary
            cropped_image = np.pad(cropped_image, ((0, 0), (x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')
            
            # extract the gfp and rfp channels to apply some vis techn
            gfp = cropped_image[gfp_channel, ...]#.compute().compute()
            rfp = cropped_image[rfp_channel, ...]#.compute().compute()
        
            # clip the images so that the contrast is more apparent
            contrast_lim_gfp = np.clip(gfp, 358, 5886)
            contrast_lim_rfp = np.clip(rfp, 480, 1300)
        
            norm_gfp = cv2.normalize(contrast_lim_gfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
            norm_rfp = cv2.normalize(contrast_lim_rfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
        
            # put the modified gfp rfp back in place
            cropped_image[gfp_channel, ...] = norm_gfp
            cropped_image[rfp_channel, ...] = norm_rfp
        
            # Create an empty RGB image with the same shape as the input image
            rgb_image = np.zeros((cropped_image.shape[1], cropped_image.shape[2], 3), dtype=np.uint16)
        
            # Assign the first channel to the green channel of the RGB image
            rgb_image[:, :, 1] = cropped_image[gfp_channel]
        
            # Assign the second channel to the red and blue channels of the RGB image to create magenta
            rgb_image[:, :, 0] = cropped_image[rfp_channel]
            rgb_image[:, :, 2] = cropped_image[rfp_channel]
            
            # scale down to 8bit
            rgb_image = np.uint8(rgb_image >> 8)
            
            if mask_outline:
                # load mask (singular)
                
                cropped_masks = segmentation[frame_number, x_start:x_end, y_start:y_end]
                
                # Pad the cropped image if necessary
                cropped_masks = np.pad(cropped_masks, ((x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')
        
                # extract only that segment
                seg_ID = cropped_masks[int(cropped_masks.shape[0] / 2), int(cropped_masks.shape[1] / 2)]
                
                if seg_ID == 0:
                    instance_mask = np.zeros((crop_size, crop_size), dtype = np.uint8)
                else:
                    instance_mask = (cropped_masks == seg_ID).astype(np.uint8)
        
                # draw outline
                contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cv2.drawContours(rgb_image, contours, -1, (0, 2 ** 8, 2 ** 8), thickness=2)  # make 8bit
        
            # # downsize image to reduce storage demands
            # rgb_image = cv2.resize(rgb_image, (rgb_image.shape[1] // 2, rgb_image.shape[0] // 2))
            
            # add labels 
            if add_labels:
                # Add text overlay
                text_color = (255, 255, 0)  # Cyan color in BGR format
                font_scale = 1.2
                font_thickness = 2
                text_thickness = 1
                font = cv2.FONT_HERSHEY_SIMPLEX
                
                # Bottom left corner text
                bottom_left_text = f"{t}hr"
                cv2.putText(rgb_image, bottom_left_text, (10, rgb_image.shape[0] - 20),
                            font, font_scale, text_color, font_thickness)
                
                # Bottom right corner text
                bottom_right_text = scale_bar_text = '20mu'
                text_size = cv2.getTextSize(bottom_right_text, font, font_scale, text_thickness)[0]
                bottom_right_text_position = (rgb_image.shape[1] - text_size[0] - 40, rgb_image.shape[0] - 20)
                cv2.putText(rgb_image, bottom_right_text, bottom_right_text_position,
                            font, font_scale, text_color, font_thickness)
                
                # Calculate line length in pixels for a 20 micrometer scale bar
                micrometer_scale_bar_length = 20
                line_length_pixels = int(micrometer_scale_bar_length / image_scale_um_per_pixel)
                
                # Draw line under bottom right text
                line_color = (255, 255, 0)  # Yellow color in BGR format
                line_thickness = 2
                line_start = (bottom_right_text_position[0], bottom_right_text_position[1] + 5)  # Adjusted by 5 pixels
                line_end = (line_start[0] + line_length_pixels, line_start[1])
                cv2.line(rgb_image, line_start, line_end,
                         line_color, line_thickness)
                
                # Top left corner text
                top_left_variable1 = ID
                cv2.putText(rgb_image, top_left_variable1, (10, 30),
                            font, font_scale, text_color, font_thickness)
                
                # Top right corner text
                mtb_value = sc_df_t['Mtb Area (µm)'].iloc[0]
                mtb_value = f"{mtb_value:.2f}mu sq"
                text_size = cv2.getTextSize(mtb_value, font, font_scale, text_thickness)[0]
                cv2.putText(rgb_image, mtb_value, (rgb_image.shape[1] - text_size[0] - 10, 30),
                            font, font_scale, text_color, font_thickness)
    
            
            rgb_stack.append(rgb_image)
    
        max_height = max(rgb_image.shape[0] for rgb_image in rgb_stack)
        max_width = max(rgb_image.shape[1] for rgb_image in rgb_stack)
        
        # Resize each image to the maximum dimensions
        resized_rgb_stack = [cv2.resize(rgb_image, (max_width, max_height)) for rgb_image in rgb_stack]
        
        # Stack the resized images
        rgb_stack = np.stack(resized_rgb_stack, axis=0)
        # Get the dimensions of the first frame
        height, width, _ = rgb_stack[0].shape
        
        # Define the frame rate (number of frames per second)
        frame_rate = len(rgb_stack) / 20  # Total frames divided by total seconds
        ## hacky fix to put outside of subdir
        output_file = os.path.join(output_dirname, f'cell_ID.{ID}.mp4')
        # output_file = f"/mnt/SYNO/macrohet_syno/manuscript/results/dt_dev_{current_date}/glimpses/{ID}_wmask.mp4"
        # Initialize VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_file, fourcc, frame_rate, (width, height))
        
        # Write each frame to the video file
        for frame in rgb_stack:
            out.write(frame)
        
        # Release the VideoWriter object
        out.release()
        
        print(f"Video {ID} saved successfully.")

               
        previous_files = set(fns)  # Update the set of previous file paths
        time.sleep(1) 

Checking for new files


[INFO][2024/04/19 05:42:19 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(3, 8).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(3, 8).h5...
[INFO][2024/04/19 05:42:56 PM] Loading segmentation (154, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (154, 6048, 6048)
[INFO][2024/04/19 05:42:56 PM] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(3, 8).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(3, 8).h5


  0%|          | 0/154 [00:00<?, ?it/s]

Video 252.3.8.ND0003 saved successfully.


[INFO][2024/04/19 06:10:10 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 3).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 3).h5...
[INFO][2024/04/19 06:10:35 PM] Loading segmentation (154, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (154, 6048, 6048)
[INFO][2024/04/19 06:10:35 PM] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 3).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 3).h5


...

[INFO][2024/04/21 12:34:40 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/PS0000/labels/macrohet_seg_model/(6, 9).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/PS0000/labels/macrohet_seg_model/(6, 9).h5...
[INFO][2024/04/21 12:35:05 PM] Loading segmentation (75, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (75, 6048, 6048)
[INFO][2024/04/21 12:35:06 PM] Closing HDF file: /mnt/SYNO/macrohet_syno/PS0000/labels/macrohet_seg_model/(6, 9).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/PS0000/labels/macrohet_seg_model/(6, 9).h5


  0%|          | 0/75 [00:00<?, ?it/s]

Video 560.6.9.PS0000 saved successfully.


  0%|          | 0/70 [00:00<?, ?it/s]

IndexError: index 75 is out of bounds for axis 0 with size 75

# Just make one glimpse

In [15]:
ID = '1946.3.10.ND0003'
fn = glob.glob(f'/mnt/SYNO/macrohet_syno/manuscript/results/dt_dev_11-04-2024/inaccurate/*{ID}*.png')[0]
fn

In [20]:
output_dirname = os.path.dirname(fn)
# set empty image array 
rgb_stack = []

# get single cell information 
sc_df = df[df['ID']==ID]

# Scale the x and y coordinates
scaled_x_coords = sc_df.x * track_scale_factor
scaled_y_coords = sc_df.y * track_scale_factor

# if (any(scaled_x_coords < boundary_buffer) or 
#     any(scaled_y_coords < boundary_buffer) or
#     any(scaled_x_coords > image_width-boundary_buffer) or 
#     any(scaled_y_coords > image_height-boundary_buffer)):
#     # Save empty text file with "boundary_case" at the beginning
#     boundary_file = os.path.join(output_dirname, f'glimpses/boundary_case_cell_ID_{ID}.txt')
#     with open(boundary_file, 'w') as f:
#         f.write("boundary_case\n")
#     continue

acq_ID = sc_df['Acquisition ID'].iloc[0] #  tuple(int(x) for x in ID.split('.')[1:3])
expt_ID = sc_df['Experiment ID'].iloc[0] #ID.split('.')[-1]
base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}'
# convert to frames
time_values = sc_df['Time (hours)']
if expt_ID != 'PS0000':
    frames = time_values*2 # if expt_ID != 'PS0000' else time_values
else:
    frames = time_values.values
# check channel enum
gfp_channel = 0 if expt_ID == 'PS0000' else 1
rfp_channel = 1 if expt_ID == 'PS0000' else 0

# load images
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
zarr_group = zarr.open(image_dir, mode='r')
images = zarr_group.images

# check if segmentation already loaded
if str(acq_ID) not in seg_fn:
    # define seg path
    if expt_ID == 'PS0000':
        seg_fn = os.path.join(base_dir, f'labels/macrohet_seg_model/{acq_ID}.h5')
    else:
        seg_fn = os.path.join(base_dir, f'labels/cpv3/{acq_ID}.h5')
    # load segmentation
    with btrack.io.HDF5FileHandler(seg_fn,
                                   'r',
                                   obj_type='obj_type_1') as reader:
        segmentation = reader.segmentation

# iterate over time values creating
for t, frame_number in tqdm(zip(time_values, frames), total = len(frames)):
    # # set to int
    # t = int(t)
    frame_number = int(frame_number)
    # get precise time point information 
    sc_df_t = sc_df[sc_df['Time (hours)'] == t]
    # Extract xy coordinates and transpose for python and area from the cell information
    y_coord, x_coord, area, t = sc_df_t.loc[:, ['x', 'y', 'Mphi Area (µm)', 'Time (hours)']].values[0]
    # Scale according to tracking shrinkage
    y_coord, x_coord = y_coord * track_scale_factor, x_coord * track_scale_factor

    if not crop_size:
        # Calculate the side length for cropping based on the square root of the area
        crop_size = int(np.sqrt(area)) * 2

    # Calculate the cropping boundaries
    x_start = int(x_coord - crop_size / 2)
    x_end = int(x_coord + crop_size / 2)
    y_start = int(y_coord - crop_size / 2)
    y_end = int(y_coord + crop_size / 2)
    # Pad the boundaries if they exceed the image dimensions
    if x_start < 0:
        x_pad = abs(x_start)
        x_start = 0
    else:
        x_pad = 0

    if x_end > images.shape[-1]:
        x_pad_end = x_end - images.shape[-1]
        x_end = images.shape[-1]
    else:
        x_pad_end = 0

    if y_start < 0:
        y_pad = abs(y_start)
        y_start = 0
    else:
        y_pad = 0

    if y_end > images.shape[-2]:
        y_pad_end = y_end - images.shape[-2]
        y_end = images.shape[-2]
    else:
        y_pad_end = 0

    # Crop the image
    cropped_image = images[frame_number, ..., x_start:x_end, y_start:y_end]
    # max projection 
    cropped_image = np.max(images[frame_number, ..., x_start:x_end, y_start:y_end], axis = 1)
    # Pad the cropped image if necessary
    cropped_image = np.pad(cropped_image, ((0, 0), (x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')
    
    # extract the gfp and rfp channels to apply some vis techn
    gfp = cropped_image[gfp_channel, ...]#.compute().compute()
    rfp = cropped_image[rfp_channel, ...]#.compute().compute()

    # clip the images so that the contrast is more apparent
    contrast_lim_gfp = np.clip(gfp, 358, 5886)
    contrast_lim_rfp = np.clip(rfp, 480, 1300)

    norm_gfp = cv2.normalize(contrast_lim_gfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
    norm_rfp = cv2.normalize(contrast_lim_rfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)

    # put the modified gfp rfp back in place
    cropped_image[gfp_channel, ...] = norm_gfp
    cropped_image[rfp_channel, ...] = norm_rfp

    # Create an empty RGB image with the same shape as the input image
    rgb_image = np.zeros((cropped_image.shape[1], cropped_image.shape[2], 3), dtype=np.uint16)

    # Assign the first channel to the green channel of the RGB image
    rgb_image[:, :, 1] = cropped_image[gfp_channel]

    # Assign the second channel to the red and blue channels of the RGB image to create magenta
    rgb_image[:, :, 0] = cropped_image[rfp_channel]
    rgb_image[:, :, 2] = cropped_image[rfp_channel]
    
    # scale down to 8bit
    rgb_image = np.uint8(rgb_image >> 8)
    
    if mask_outline:
        # load mask (singular)
        
        cropped_masks = segmentation[frame_number, x_start:x_end, y_start:y_end]
        
        # Pad the cropped image if necessary
        cropped_masks = np.pad(cropped_masks, ((x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')

        # extract only that segment
        seg_ID = cropped_masks[int(cropped_masks.shape[0] / 2), int(cropped_masks.shape[1] / 2)]
        
        if seg_ID == 0:
            instance_mask = np.zeros((crop_size, crop_size), dtype = np.uint8)
        else:
            instance_mask = (cropped_masks == seg_ID).astype(np.uint8)

        # draw outline
        contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(rgb_image, contours, -1, (0, 2 ** 8, 2 ** 8), thickness=2)  # make 8bit

    # # downsize image to reduce storage demands
    # rgb_image = cv2.resize(rgb_image, (rgb_image.shape[1] // 2, rgb_image.shape[0] // 2))
    
    # add labels 
    if add_labels:
        # Add text overlay
        text_color = (255, 255, 0)  # Cyan color in BGR format
        font_scale = 1.2
        font_thickness = 2
        text_thickness = 1
        font = cv2.FONT_HERSHEY_SIMPLEX
        
        # Bottom left corner text
        bottom_left_text = f"{t}hr"
        cv2.putText(rgb_image, bottom_left_text, (10, rgb_image.shape[0] - 20),
                    font, font_scale, text_color, font_thickness)
        
        # Bottom right corner text
        bottom_right_text = scale_bar_text = '20mu'
        text_size = cv2.getTextSize(bottom_right_text, font, font_scale, text_thickness)[0]
        bottom_right_text_position = (rgb_image.shape[1] - text_size[0] - 40, rgb_image.shape[0] - 20)
        cv2.putText(rgb_image, bottom_right_text, bottom_right_text_position,
                    font, font_scale, text_color, font_thickness)
        
        # Calculate line length in pixels for a 20 micrometer scale bar
        micrometer_scale_bar_length = 20
        line_length_pixels = int(micrometer_scale_bar_length / image_scale_um_per_pixel)
        
        # Draw line under bottom right text
        line_color = (255, 255, 0)  # Yellow color in BGR format
        line_thickness = 2
        line_start = (bottom_right_text_position[0], bottom_right_text_position[1] + 5)  # Adjusted by 5 pixels
        line_end = (line_start[0] + line_length_pixels, line_start[1])
        cv2.line(rgb_image, line_start, line_end,
                 line_color, line_thickness)
        
        # Top left corner text
        top_left_variable1 = ID
        cv2.putText(rgb_image, top_left_variable1, (10, 30),
                    font, font_scale, text_color, font_thickness)
        
        # Top right corner text
        mtb_value = sc_df_t['Mtb Area (µm)'].iloc[0]
        mtb_value = f"{mtb_value:.2f}mu sq"
        text_size = cv2.getTextSize(mtb_value, font, font_scale, text_thickness)[0]
        cv2.putText(rgb_image, mtb_value, (rgb_image.shape[1] - text_size[0] - 10, 30),
                    font, font_scale, text_color, font_thickness)

    
    rgb_stack.append(rgb_image)

max_height = max(rgb_image.shape[0] for rgb_image in rgb_stack)
max_width = max(rgb_image.shape[1] for rgb_image in rgb_stack)

# Resize each image to the maximum dimensions
resized_rgb_stack = [cv2.resize(rgb_image, (max_width, max_height)) for rgb_image in rgb_stack]

# Stack the resized images
rgb_stack = np.stack(resized_rgb_stack, axis=0)
# Get the dimensions of the first frame
height, width, _ = rgb_stack[0].shape

# Define the frame rate (number of frames per second)
frame_rate = len(rgb_stack) / 20  # Total frames divided by total seconds
## hacky fix to put outside of subdir
output_file = os.path.join(output_dirname, f'glimpses/cell_ID.{ID}.mp4')
# output_file = f"/mnt/SYNO/macrohet_syno/manuscript/results/dt_dev_{current_date}/glimpses/{ID}_wmask.mp4"
# Initialize VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, frame_rate, (width, height))

# Write each frame to the video file
for frame in rgb_stack:
    out.write(frame)

# Release the VideoWriter object
out.release()

print(f"Video {ID} saved successfully.")

[INFO][2024/04/15 05:17:45 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(3, 10).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(3, 10).h5...
[INFO][2024/04/15 05:18:06 PM] Loading segmentation (154, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (154, 6048, 6048)
[INFO][2024/04/15 05:18:07 PM] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(3, 10).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(3, 10).h5


  0%|          | 0/118 [00:00<?, ?it/s]

Video 1946.3.10.ND0003 saved successfully.
